This program searches a given directory for all *.spe files in subfolders, then lists their characteristics and formulates a plan for proceeding with data reduction.

In [1]:
cd '..'

/Users/keatonb/github/reduceSPE


In [2]:
# Imports.
# Standard libraries.
from __future__ import absolute_import, division, print_function
import os
import csv
import sys
from glob import glob
import datetime as dt
import dateutil.parser
import numpy as np
import matplotlib.pyplot as plt
# Installed packages.
from astropy.io import fits
import pandas as pd
from pandas import Series, DataFrame
from bs4 import BeautifulSoup
# Local modules.
import read_spe

%matplotlib inline

/Users/keatonb/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [3]:
#directory to search
directory = '/Users/keatonb/Observing/ProEMData/20150913/'
spefiles = glob(directory+'*/*.spe')
spefiles = [spefile[len(directory):] for spefile in spefiles]
splitfiles = [spefile.split('/') for spefile in spefiles]
subdir = [splitfile[0] for splitfile in splitfiles]
filename = [splitfile[1] for splitfile in splitfiles]
files = {'subdir':subdir,'filename':filename}
s = DataFrame(files,columns=['subdir','filename','type','hasfooter','timestart','numframes','exptime','shutter','xbinning','ybinning','triggerresponse'])
s.name = directory.split('/')[-2]

#loop through files and get information
for i in s.index:
    spe = read_spe.File(directory+s.subdir[i]+'/'+s.filename[i])
    s.numframes[i] = spe.get_num_frames()
    if hasattr(spe, 'footer_metadata'):
        s.hasfooter[i] = True
        footer_metadata = BeautifulSoup(spe.footer_metadata, "xml")
        ts_begin = footer_metadata.find(name='TimeStamp', event='ExposureStarted').attrs['absoluteTime']
        dt_begin = dateutil.parser.parse(ts_begin)
        s.timestart[i] = str(dt_begin.isoformat())[:-6]
        s.xbinning[i] = footer_metadata.find(name="SensorMapping").attrs['xBinning']
        s.ybinning[i] = footer_metadata.find(name="SensorMapping").attrs['yBinning']
        s.triggerresponse[i] = footer_metadata.find(name='TriggerResponse').text
        s.exptime[i] = float(footer_metadata.find(name='ExposureTime').text)/1000.
        s.shutter[i] = footer_metadata.find(name='Mode').text
    else:
        s.hasfooter[i] = False
    spe.close()

#identify image type
s.type[(s.exptime == 0) & (s.shutter == "AlwaysClosed")] = 'bias'
s.type[(s.exptime != 0) & (s.shutter == "AlwaysClosed")] = 'dark'
s.type[['Open' in f for f in s.shutter]] = 'target'
s.type[['Open' in f for f in s.shutter] and ['flat' in f.lower() for f in s.filename] and ['flat' in f.lower() for f in s.subdir]] = 'flat'
s

/Users/keatonb/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,subdir,filename,type,hasfooter,timestart,numframes,exptime,shutter,xbinning,ybinning,triggerresponse
0,bias,bias.spe,bias,True,2015-09-12T22:04:33.466378,91,0,AlwaysClosed,4,4,ReadoutPerTrigger
1,dark,dark_10s.spe,dark,True,2015-09-12T23:18:03.762559,61,9.998,AlwaysClosed,4,4,ReadoutPerTrigger
2,dark,dark_15s.spe,dark,True,2015-09-12T23:28:53.278337,61,14.998,AlwaysClosed,4,4,ReadoutPerTrigger
3,dark,dark_20s.spe,dark,True,2015-09-13T00:02:53.825376,61,19.998,AlwaysClosed,4,4,ReadoutPerTrigger
4,dark,dark_30s.spe,dark,True,2015-09-13T00:38:34.413563,61,29.998,AlwaysClosed,4,4,ReadoutPerTrigger
5,dark,dark_3s.spe,dark,True,2015-09-12T22:08:51.438287,91,2.998,AlwaysClosed,4,4,ReadoutPerTrigger
6,dark,dark_5s.spe,dark,True,2015-09-12T23:06:28.858678,61,4.998,AlwaysClosed,4,4,ReadoutPerTrigger
7,dome_flat,dome_flat_BG40_3s.spe,flat,True,2015-09-13T01:44:24.646313,91,2.998,AlwaysOpen,4,4,ReadoutPerTrigger
8,GD244,GD244.spe,target,True,2015-09-13T09:13:52.772723,1867,4.998,AlwaysOpen,4,4,ReadoutPerTrigger
9,SDSSJ2237+0522,SDSSJ2237+0522.spe,target,True,2015-09-13T02:37:51.892802,1093,19.998,AlwaysOpen,4,4,ReadoutPerTrigger


In [4]:
#Split by type
target=s.ix[s.type.values == "target"]
#target.index=range(target.shape[0])
target.name="target"
target

,subdir,filename,type,hasfooter,timestart,numframes,exptime,shutter,xbinning,ybinning,triggerresponse
8,GD244,GD244.spe,target,True,2015-09-13T09:13:52.772723,1867,4.998,AlwaysOpen,4,4,ReadoutPerTrigger
9,SDSSJ2237+0522,SDSSJ2237+0522.spe,target,True,2015-09-13T02:37:51.892802,1093,19.998,AlwaysOpen,4,4,ReadoutPerTrigger


In [5]:
dark=s.ix[s.type.values == "dark"]
#dark.index=range(dark.shape[0])
dark.name="dark"
dark

,subdir,filename,type,hasfooter,timestart,numframes,exptime,shutter,xbinning,ybinning,triggerresponse
1,dark,dark_10s.spe,dark,True,2015-09-12T23:18:03.762559,61,9.998,AlwaysClosed,4,4,ReadoutPerTrigger
2,dark,dark_15s.spe,dark,True,2015-09-12T23:28:53.278337,61,14.998,AlwaysClosed,4,4,ReadoutPerTrigger
3,dark,dark_20s.spe,dark,True,2015-09-13T00:02:53.825376,61,19.998,AlwaysClosed,4,4,ReadoutPerTrigger
4,dark,dark_30s.spe,dark,True,2015-09-13T00:38:34.413563,61,29.998,AlwaysClosed,4,4,ReadoutPerTrigger
5,dark,dark_3s.spe,dark,True,2015-09-12T22:08:51.438287,91,2.998,AlwaysClosed,4,4,ReadoutPerTrigger
6,dark,dark_5s.spe,dark,True,2015-09-12T23:06:28.858678,61,4.998,AlwaysClosed,4,4,ReadoutPerTrigger


In [6]:
flat=s.ix[s.type.values == "flat"]
flat.name="flat"
flat

,subdir,filename,type,hasfooter,timestart,numframes,exptime,shutter,xbinning,ybinning,triggerresponse
7,dome_flat,dome_flat_BG40_3s.spe,flat,True,2015-09-13T01:44:24.646313,91,2.998,AlwaysOpen,4,4,ReadoutPerTrigger


In [41]:
#Print what darks need to be subtracted from object,flat frames
for i in target.index:
    this = s.ix[i]
    print(' - What to do with ' + this.subdir+'/'+this.filename + '?')
    exp = this.exptime
    #Is there a corresponding dark exposure time?
    if np.where(exp == dark.exptime)[0].size > 0: #yes, at least on dark matches, use first
        
    #print(exp)
    #print([(exp == expt) for expt in dark.exptime])
    #print(np.where(exp == dark.exptime)[0].size)
    #print(exp in dark.exptime)
    #if exp in dark.exptime:
        thisdark=dark.ix[dark.index[np.where(dark.exptime == this.exptime)[0][0]]]
        #print(exptime)
        #print(thisdark.exptime)
        #print(np.where(dark.exptime == this.exptime)[0][0])
        print('Subtract ' + thisdark.subdir + '/'+thisdark.filename + ' from ' + this.subdir+'/'+this.filename)
        
#Loop through flat frames:
for i in flat.index:
    this = s.ix[i]
    exp = this.exptime
    print(' - What to do with ' + this.subdir+'/'+this.filename + '?')
    if exp in dark.exptime:
        thisdark=dark.ix[dark.index[np.where(dark.exptime == this.exptime)[0][0]]]
        print('Subtract ' + thisdark.subdir + '/'+thisdark.filename + ' from ' + this.subdir+'/'+this.filename)

 - What to do with GD244/GD244.spe?
Subtract dark/dark_5s.spe from GD244/GD244.spe
 - What to do with SDSSJ2237+0522/SDSSJ2237+0522.spe?
Subtract dark/dark_20s.spe from SDSSJ2237+0522/SDSSJ2237+0522.spe
 - What to do with dome_flat/dome_flat_BG40_3s.spe?
Subtract dark/dark_3s.spe from dome_flat/dome_flat_BG40_3s.spe
